In [1]:
from datasets import load_dataset
import json
print("✓ Ready")

✓ Ready


In [2]:
# SUPER SMALL - 2000 samples
SAMPLE_SIZE = 2000

print(f"Loading {SAMPLE_SIZE} samples...")
ds = load_dataset("salmankhanpm/Corpous_Telugu_Tokenizer", streaming=True, split="train")

texts = []
for i, example in enumerate(ds):
    if i >= SAMPLE_SIZE:
        break
    text = list(example.values())[0]
    if text and isinstance(text, str) and text.strip():
        texts.append(text.strip())

corpus = " ".join(texts)
tokens = list(corpus.encode("utf-8"))

print(f"✓ {len(texts)} texts, {len(tokens):,} bytes")

Loading 2000 samples...


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:104: UserWarning: 
Error while fetching `HF_TOKEN` secret value from your vault: 'Requesting secret HF_TOKEN timed out. Secrets can only be fetched when running from the Colab UI.'.
You are not authenticated with the Hugging Face Hub in this notebook.
If the error persists, please let us know by opening an issue on GitHub (https://github.com/huggingface/huggingface_hub/issues/new).
  warnings.warn(


README.md:   0%|          | 0.00/298 [00:00<?, ?B/s]

Resolving data files:   0%|          | 0/82 [00:00<?, ?it/s]

Resolving data files:   0%|          | 0/82 [00:00<?, ?it/s]

✓ 2000 texts, 4,776,832 bytes


In [3]:
# BPE functions
def get_stats(ids):
    counts = {}
    for pair in zip(ids, ids[1:]):
        counts[pair] = counts.get(pair, 0) + 1
    return counts

def merge(ids, pair, idx):
    newids = []
    i = 0
    while i < len(ids):
        if i < len(ids) - 1 and ids[i] == pair[0] and ids[i+1] == pair[1]:
            newids.append(idx)
            i += 2
        else:
            newids.append(ids[i])
            i += 1
    return newids

print("✓ Functions ready")

✓ Functions ready


In [4]:
# TRAIN - Should be FAST now
vocab_size = 5000
num_merges = vocab_size - 256

print(f"Training {num_merges} merges...")
print("Should take 2-3 minutes\n")

ids = list(tokens)
merges = {}

import time
start = time.time()

for i in range(num_merges):
    stats = get_stats(ids)
    pair = max(stats, key=stats.get)
    idx = 256 + i
    ids = merge(ids, pair, idx)
    merges[pair] = idx

    if (i + 1) % 100 == 0:
        elapsed = time.time() - start
        rate = (i + 1) / elapsed
        eta = (num_merges - i - 1) / rate / 60
        print(f"Merge {i+1:4d}/{num_merges} | {elapsed:.1f}s elapsed | ETA: {eta:.1f} min")

total_time = time.time() - start
print(f"\n✓ Done in {total_time/60:.1f} minutes!")
print(f"Vocab: {256 + len(merges)}")

Training 4744 merges...
Should take 2-3 minutes

Merge  100/4744 | 91.4s elapsed | ETA: 70.7 min
Merge  200/4744 | 152.6s elapsed | ETA: 57.8 min
Merge  300/4744 | 206.2s elapsed | ETA: 50.9 min
Merge  400/4744 | 256.8s elapsed | ETA: 46.5 min
Merge  500/4744 | 305.2s elapsed | ETA: 43.2 min
Merge  600/4744 | 352.5s elapsed | ETA: 40.6 min
Merge  700/4744 | 397.8s elapsed | ETA: 38.3 min
Merge  800/4744 | 443.6s elapsed | ETA: 36.4 min
Merge  900/4744 | 488.3s elapsed | ETA: 34.8 min
Merge 1000/4744 | 532.3s elapsed | ETA: 33.2 min
Merge 1100/4744 | 576.0s elapsed | ETA: 31.8 min
Merge 1200/4744 | 619.0s elapsed | ETA: 30.5 min
Merge 1300/4744 | 661.8s elapsed | ETA: 29.2 min
Merge 1400/4744 | 705.0s elapsed | ETA: 28.1 min
Merge 1500/4744 | 748.0s elapsed | ETA: 27.0 min
Merge 1600/4744 | 792.4s elapsed | ETA: 26.0 min
Merge 1700/4744 | 834.8s elapsed | ETA: 24.9 min
Merge 1800/4744 | 877.9s elapsed | ETA: 23.9 min
Merge 1900/4744 | 919.4s elapsed | ETA: 22.9 min
Merge 2000/4744 | 961

In [5]:
# Build vocab
vocab = {idx: bytes([idx]) for idx in range(256)}
for (p0, p1), idx in merges.items():
    vocab[idx] = vocab[p0] + vocab[p1]

print(f"✓ Vocab: {len(vocab)}")

✓ Vocab: 5000


In [6]:
# Encode/Decode
def decode(ids):
    return b"".join(vocab[idx] for idx in ids).decode("utf-8", errors="replace")

def encode(text):
    tokens = list(text.encode("utf-8"))
    while len(tokens) >= 2:
        stats = get_stats(tokens)
        pair = min(stats, key=lambda p: merges.get(p, float("inf")))
        if pair not in merges:
            break
        idx = merges[pair]
        tokens = merge(tokens, pair, idx)
    return tokens

print("✓ Encode/decode ready")

✓ Encode/decode ready


In [7]:
# Test
tests = ["నమస్కారం", "తెలుగు భాష", "నేను తెలుగు నేర్చుకుంటున్నాను"]

print("TESTS:")
for text in tests:
    enc = encode(text)
    dec = decode(enc)
    print(f"{text} → {len(enc)} tokens, Match: {text==dec}")

TESTS:
నమస్కారం → 4 tokens, Match: True
తెలుగు భాష → 5 tokens, Match: True
నేను తెలుగు నేర్చుకుంటున్నాను → 8 tokens, Match: True


In [8]:
# Stats
sample = texts[:300]
total_chars = sum(len(t.replace(' ', '')) for t in sample)
total_tokens = sum(len(encode(t)) for t in sample)
compression_ratio = total_chars / total_tokens

print(f"\nVocab: {len(vocab)} ✓")
print(f"Compression: {compression_ratio:.2f}")
print(f"Status: {'PASS' if len(vocab) >= 5000 else 'ADJUST'}")


Vocab: 5000 ✓
Compression: 3.24
Status: PASS


In [9]:
# Save
with open('telugu_merges.json', 'w') as f:
    json.dump({str(k): v for k, v in merges.items()}, f)

with open('telugu_vocab.json', 'w') as f:
    json.dump({k: v.hex() for k, v in vocab.items()}, f)

print("✓ Saved tokenizer")

✓ Saved tokenizer


In [10]:
# README
readme = f"""---
language: te
tags: [tokenizer, bpe, telugu]
---

# Telugu BPE Tokenizer

Byte Pair Encoding tokenizer for Telugu.

**Stats:**
- Vocab: {len(vocab):,}
- Compression: {compression_ratio:.2f}x
- Training: {len(texts):,} texts

**Usage:**

```python
import json

# Load tokenizer
with open('telugu_merges.json') as f:
    merges = {{eval(k): v for k, v in json.load(f).items()}}
with open('telugu_vocab.json') as f:
    vocab = {{int(k): bytes.fromhex(v) for k, v in json.load(f).items()}}

def get_stats(ids):
    counts = {{}}
    for pair in zip(ids, ids[1:]):
        counts[pair] = counts.get(pair, 0) + 1
    return counts

def merge(ids, pair, idx):
    newids = []
    i = 0
    while i < len(ids):
        if i < len(ids) - 1 and ids[i] == pair[0] and ids[i+1] == pair[1]:
            newids.append(idx)
            i += 2
        else:
            newids.append(ids[i])
            i += 1
    return newids

def encode(text):
    tokens = list(text.encode("utf-8"))
    while len(tokens) >= 2:
        stats = get_stats(tokens)
        pair = min(stats, key=lambda p: merges.get(p, float("inf")))
        if pair not in merges:
            break
        tokens = merge(tokens, pair, merges[pair])
    return tokens

def decode(ids):
    return b"".join(vocab[i] for i in ids).decode("utf-8", errors="replace")

# Example
text = "నమస్కారం"
print(decode(encode(text)))
```

**Examples:**
- నమస్కారం → {len(encode('నమస్కారం'))} tokens
- తెలుగు భాష → {len(encode('తెలుగు భాష'))} tokens
"""

with open('README.md', 'w', encoding='utf-8') as f:
    f.write(readme)
print("✓ README")

✓ README


In [14]:
# HuggingFace login
from huggingface_hub import login
login()

In [15]:
# Upload
from huggingface_hub import HfApi

YOUR_USERNAME = "vnaren13"  # CHANGE THIS!
REPO = "telugu-bpe-tokenizer"

api = HfApi()
api.create_repo(f"{YOUR_USERNAME}/{REPO}", exist_ok=True)

for file in ['telugu_merges.json', 'telugu_vocab.json', 'README.md']:
    api.upload_file(
        path_or_fileobj=file,
        path_in_repo=file,
        repo_id=f"{YOUR_USERNAME}/{REPO}"
    )
    print(f"✓ {file}")

print(f"\n🎉 https://huggingface.co/{YOUR_USERNAME}/{REPO}")

✓ telugu_merges.json
✓ telugu_vocab.json
✓ README.md

🎉 https://huggingface.co/vnaren13/telugu-bpe-tokenizer


In [16]:
# BONUS: Gradio frontend
import gradio as gr

def tokenize_ui(text):
    if not text.strip():
        return "Enter text!", "", ""
    enc = encode(text)
    dec = decode(enc)
    stats = f"Chars: {len(text)} | Tokens: {len(enc)} | Compression: {len(text)/len(enc):.2f}x"
    return stats, str(enc[:30]) + "...", dec

demo = gr.Interface(
    fn=tokenize_ui,
    inputs=gr.Textbox(label="Telugu Text", lines=3),
    outputs=[
        gr.Textbox(label="Stats"),
        gr.Textbox(label="Tokens"),
        gr.Textbox(label="Decoded")
    ],
    title="Telugu BPE Tokenizer",
    examples=[["నమస్కారం"], ["తెలుగు భాష"]]
)

demo.launch(share=True)

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://13082b7aad2033bae1.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
